### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [16]:
num_schools = school_data["school_name"].count()
student_tot = school_data_complete["student_name"].count()
budg_tot = school_data["budget"].sum()
avg_math = round(school_data_complete["math_score"].mean(),2)
avg_read = round(school_data_complete["reading_score"].mean(),2)
pass_bin = [0,70,100]
pass_labels = ["Fail","Pass"]
school_data_complete["read_res"] = pd.cut(school_data_complete["reading_score"], pass_bin, labels=pass_labels)
school_data_complete["math_res"] = pd.cut(school_data_complete["math_score"], pass_bin, labels=pass_labels)
read_pass_count = (school_data_complete["read_res"] == "Pass").sum()
read_pass_perc = f"{round((read_pass_count / student_tot) *100, 2)}%"
math_pass_count = (school_data_complete["math_res"] == "Pass").sum()
math_pass_perc = f"{round((math_pass_count / student_tot) *100, 2)}%"
combo_pass_count = ((school_data_complete["read_res"] == "Pass") & (school_data_complete["math_res"] == "Pass")).sum()
combo_pass_perc = f"{round((combo_pass_count / student_tot) *100, 2)}%"
dist_summ_df = pd.DataFrame([[num_schools,student_tot,budg_tot,avg_math,math_pass_perc,avg_read,read_pass_perc,combo_pass_perc]], columns=["Total Schools","Total Students","Total Budget","Average Math Score","% Passing Math","Average Reading Score","% Passing Reading","% Overall Passing"])


dist_summ_df

,Total Schools,Total Students,Total Budget,Average Math Score,% Passing Math,Average Reading Score,% Passing Reading,% Overall Passing
0,15,39170,24649428,78.99,72.39%,81.88,82.97%,60.8%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [142]:
#school_summ = school_data_complete[["school_name", "type", "student_name", "budget", "math_score", "reading_score"]]
group_schools_2 = school_data_complete.set_index("school_name")
group_schools = school_data_complete.groupby("school_name")

group_sch_avgs = group_schools[["math_score", "reading_score"]].mean()
group_sch_stu_count = group_schools.agg(Total_Students = pd.NamedAgg(column="student_name", aggfunc="count"))
group_sch_set_vars = group_schools_2.loc[ : , ["type", "budget"]]
mer_sch_summ = pd.merge(pd.merge(group_sch_avgs,group_sch_stu_count, on="school_name"),group_sch_set_vars, on="school_name", how="left").drop_duplicates()
mer_sch_summ["Per Student Budget"] = mer_sch_summ["budget"]/mer_sch_summ["Total_Students"]

#All lines for creating results (except for overall)
df_by_school = school_data_complete.groupby(['school_name'])
school_type = df_by_school['type'].first()
school_budget = df_by_school['budget'].first()
school_size = df_by_school['size'].first()
avg_math = df_by_school['math_score'].mean()
avg_read = df_by_school['reading_score'].mean()
bug_per_stud = school_budget/school_size
read_pass = ((df_by_school['read_res'].apply(lambda x : x.str.contains('Pass').sum()))/school_size) * 100
math_pass = ((df_by_school['math_res'].apply(lambda x : x.str.contains('Pass').sum()))/school_size) * 100
combo_count = (school_data_complete.query("math_res == 'Pass' & read_res == 'Pass'").groupby('school_name').size()/school_size) * 100

school_sum = pd.concat([school_type,school_size,school_budget,bug_per_stud,avg_math,avg_read,math_pass,read_pass,combo_count], axis='columns', sort=False)
school_sum.rename(columns={'type':'School Type', 'size': 'Total Students', 'budget': 'Total School Budget', 0 : 'Per Student Budget','math_score':'Average Math Score','reading_score':'Average Reading Score',2:'% Passing Reading',1:'% Passing Math',3:'% Passing Overall'}, inplace=True)

school_sum

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
school_name,,,,,,,,,
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,64.630225,79.300643,51.145498
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,89.558665,93.864370,84.015070
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,63.750424,78.433367,49.915226
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,65.753925,77.510040,51.296093
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,89.713896,93.392371,83.651226
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,64.746494,78.187702,50.161812
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,90.632319,92.740047,84.074941
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,63.318478,78.813850,49.914296
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,63.852132,78.281874,49.800462


In [146]:
## Top Performing Schools (By % Overall Passing)
school_sum.sort_values(by='% Passing Overall', ascending=False).head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
school_name,,,,,,,,,
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,90.932983,93.254490,84.888305
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,91.683992,92.203742,84.823285
Wright High School,Charter,1800,1049400,583.0,83.682222,83.955000,90.277778,93.444444,84.444444
Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,90.214067,92.905199,84.281346
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,90.632319,92.740047,84.074941


* Sort and display the top five performing schools by % overall passing.

## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [147]:
school_sum.sort_values(by='% Passing Overall', ascending=True).head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
school_name,,,,,,,,,
Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,64.066017,77.744436,49.437359
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,63.852132,78.281874,49.800462
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,63.318478,78.813850,49.914296
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,63.750424,78.433367,49.915226
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,64.746494,78.187702,50.161812


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [192]:
grade9 = school_data_complete.query("grade == '9th'").groupby('school_name').math_score.mean().rename("9th")
grade10 = school_data_complete.query("grade == '10th'").groupby('school_name').math_score.mean().rename("10th")
grade11 = school_data_complete.query("grade == '11th'").groupby('school_name').math_score.mean().rename("11th")
grade12 = school_data_complete.query("grade == '12th'").groupby('school_name').math_score.mean().rename("12th")
allmathGrades = pd.concat([grade9, grade10, grade11, grade12], axis='columns', sort=False)

allmathGrades

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [193]:
grade9 = school_data_complete.query("grade == '9th'").groupby('school_name').reading_score.mean().rename("9th")
grade10 = school_data_complete.query("grade == '10th'").groupby('school_name').reading_score.mean().rename("10th")
grade11 = school_data_complete.query("grade == '11th'").groupby('school_name').reading_score.mean().rename("11th")
grade12 = school_data_complete.query("grade == '12th'").groupby('school_name').reading_score.mean().rename("12th")
allreadingGrades = pd.concat([grade9, grade10, grade11, grade12], axis='columns', sort=False)

allreadingGrades

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

## Scores by School Size

* Perform the same operations as above, based on school size.

## Scores by School Type

* Perform the same operations as above, based on school type